In [21]:
import production as p
import os
import inspect
import sys
import pandas as pd

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step1: Read the LLM-ready data

In [22]:
df_user_data,neutral_sentences = p.read_input_data()

In [23]:
df_user_data.head(3)

,username,messageID,original
49,U0,29,"I have a younger sister, 22 now, and a mother ..."
47,U0,20,I am gonna sound very young when I tell you ab...
46,U0,2,So there is one that we have in common and tha...


In [24]:
neutral_sentences

,idSentence,sentences
0,0,I’m all about that food. I usually kick off th...
1,1,Just getting my vitamins in at the school cant...
2,2,De Pizzabakkers sell this vegan pizza with che...
3,3,I entered the world of vegan foods lately. Nex...
4,4,This vegan fried chicken from KFC is on the sp...
5,5,Just having this vegan hotdog from the school ...
6,6,"Just had my first vegan cake at ""groene bakker..."
7,7,This vegan chocolate is on point. Its with oat...
8,8,I'm all about fast food. I'm having some fries...
9,9,"I went to the ""Groene Burger"" fast food restau..."


### Step 2 LLM TST

In [30]:
# This calls Mistral API
df_llm_tst = p.llm_tst(df_user_data,neutral_sentences[0:2])
df_llm_tst.head(3)

Processing users: 100%|██████████| 10/10 [00:29<00:00,  2.97s/it]


,username,id_neutral_sentence,neutral_sentence,tst_id,tst_sentence,explanation,llm_tst,query,model,prompt_tokens,completion_tokens,object,promptID,timestamp,output_run
0,U0,0,I’m all about that food. I usually kick off th...,U020240605134622,"I'm totally into food, you know? I start my we...",The conversational style of person X includes...,"tst\_sentence: ""I'm totally into food, you kno...",\n[INST]You are an expert in text style transf...,mistral-small,325,148,chat.completion,2,20240605134622,run_20240605134620
1,U0,1,Just getting my vitamins in at the school cant...,U020240605134623,So I'm just getting my vitamins at the school ...,"Person X uses conversational language, includ...","tst\_sentence: ""So I'm just getting my vitamin...",\n[INST]You are an expert in text style transf...,mistral-small,329,122,chat.completion,2,20240605134623,run_20240605134620
2,U1,0,I’m all about that food. I usually kick off th...,U120240605134624,"No worries about food, I've got it covered! Ju...",Person X's conversational style often include...,"tst\_sentence: ""No worries about food, I've go...",\n[INST]You are an expert in text style transf...,mistral-small,322,141,chat.completion,2,20240605134624,run_20240605134620


### Step 3 LLM Evaluate

In [28]:
# In case you wanna read other csv files
# df_llm_tst = pd.read_csv('f6_llm_tst_data/run_20240604131627/run_20240604131627_tst_postprocess.csv')

In [31]:
df_eval = p.llm_evl(df_llm_tst)
df_eval.head(3)

Processing sentences: 100%|██████████| 20/20 [01:41<00:00,  5.10s/it]

Exception at index: 73  
 with value: score_sentiment=3
explanation: The text expresses excitement and enthusiasm about food, specifically plant-based options. There are no negative words or phrases indicating dissatisfaction or disappointment. The focus on healthy food choices also contributes to a positive sentiment.


,tst_id,tst_sentence,username,id_neutral_sentence,eval_score_formality,timestamp_score_formality,eval_explanation_formality,eval_score_descriptiveness,timestamp_score_descriptiveness,eval_explanation_descriptiveness,...,eval_explanation_emotionality,eval_score_sentiment,timestamp_score_sentiment,eval_explanation_sentiment,eval_score_fluency,timestamp_score_fluency,eval_score_comprehensibility,timestamp_score_comprehensibility,eval_explanation_fluency_comprehensibility,output_run
0,U020240605134622,"I'm totally into food, you know? I start my we...",U0,0,1,20240605134651,The text contains multiple signs of informalit...,2,20240605134652,"The text provides some descriptive elements, s...",...,The text is predominantly emotional due to the...,3,20240605134654,The text expresses a positive sentiment toward...,2,20240605134655,3,20240605134655,"The text is written in English, so the ""score\...",run_20240605134620
1,U020240605134623,So I'm just getting my vitamins at the school ...,U0,1,1,20240605134656,The text contains multiple features of inform...,2,20240605134657,"The text contains some descriptive elements, s...",...,The text is predominantly emotional because it...,2,20240605134659,The text is neutral as it is describing a mund...,2,20240605134701,2,20240605134701,"The text is written in English, so it gets a ...",run_20240605134620
2,U120240605134624,"No worries about food, I've got it covered! Ju...",U1,0,2,20240605134702,The text contains some elements of informality...,2,20240605134703,The text provides some descriptive details abo...,...,The text is predominantly emotional due to the...,3,20240605134705,The text has a positive sentiment due to the o...,2,20240605134707,3,20240605134707,"The text is written entirely in English, so it...",run_20240605134620
